### Image Blending Using Pyramids

In [1]:
import cv2
import numpy as np

In [2]:
apple = cv2.imread('./data/apple.jpg')
orange = cv2.imread('./data/orange.jpg')
print(apple.shape)
print(orange.shape)

(512, 512, 3)
(512, 512, 3)


In [6]:
#half and half method
apple_orange = np.hstack((apple[:, :256], orange[:, 256:]))
#cuts each image in half and merged the two halfs together. 
# it works but it has a sharp ege between the two halves.

In [7]:
#blending method using image pyramids
#five steps required to blend with pyramids
#1. load the two images 
#2. find the gaussian pyramds for apple and orange (we will use 6 pyramids)
#3. from gaussian pyramids, find their laplacian pyramids
#4. now joing the left half and right half in each level of laplacian pyramids
#5. finally, from this join image pypramids, reconstruct the origianl image.

#1.
apple = cv2.imread('./data/apple.jpg')
orange = cv2.imread('./data/orange.jpg')

#2.
apple_copy = apple.copy()
gp_apple = [apple_copy]
for i in range(6):
    apple_copy = cv2.pyrDown(apple_copy)
    gp_apple.append(apple_copy)

orange_copy = orange.copy()
gp_orange = [orange_copy]
for i in range(6):
    orange_copy = cv2.pyrDown(orange_copy)
    gp_orange.append(orange_copy)
    
#3.
apple_copy = gp_apple[5]
lp_apple = [apple_copy]
for i in range(5,0,-1):
    gaussian_extended = cv2.pyrUp(gp_apple[i])
    laplacian = cv2.subtract(gp_apple[i-1], gaussian_extended)
    lp_apple.append(laplacian)
    
orange_copy = gp_orange[5]
lp_orange = [orange_copy]
for i in range(5,0,-1):
    gaussian_extended = cv2.pyrUp(gp_orange[i])
    laplacian = cv2.subtract(gp_orange[i-1], gaussian_extended)
    lp_orange.append(laplacian)
    
#4.
apple_orange_pyramid = []
n = 0
for apple_lap, orange_lap in zip(lp_apple, lp_orange):
    n += 1
    cols, rows, ch = apple_lap.shape
    laplacian = np.hstack((apple_lap[:,0:int(cols/2)], orange_lap[:,int(cols/2):]))
    apple_orange_pyramid.append(laplacian)
    
#5.
apple_orange_reconstruct = apple_orange_pyramid[0]
for i in range(1,6):
    apple_orange_reconstruct = cv2.pyrUp(apple_orange_reconstruct)
    apple_orange_reconstruct = cv2.add(apple_orange_pyramid[i], apple_orange_reconstruct)

In [8]:
cv2.imshow('apple', apple)
cv2.imshow('orange', orange)
cv2.imshow('apple_orange', apple_orange)
cv2.imshow('ao reconstruct', apple_orange_reconstruct)
cv2.waitKey(0)
cv2.destroyAllWindows()